## 层和块
首先，我们回顾一下多层感知机

In [12]:
import torch
from torch import nn #使用 torch 中 nn 这个 module
from torch.nn import functional as F # 在 nn 这个 module 中 将 functional 这个 module 弄进来
 
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) #线性层 + ReLU + 线性层

X = torch.rand(2, 20) #随机生成一个 2 * 20 的二维张量（矩阵）
X, net(X) # 将 X 输入到自定义的网络 net 中得到 output

(tensor([[0.3121, 0.7391, 0.3290, 0.6910, 0.3897, 0.4597, 0.2299, 0.9887, 0.7315,
          0.2282, 0.0174, 0.1314, 0.5085, 0.9960, 0.6183, 0.5059, 0.4138, 0.3307,
          0.6341, 0.6661],
         [0.1095, 0.5525, 0.4074, 0.5402, 0.5293, 0.5163, 0.6966, 0.4817, 0.0983,
          0.2525, 0.6366, 0.1356, 0.0369, 0.2722, 0.2414, 0.5731, 0.9416, 0.9487,
          0.1678, 0.6750]]),
 tensor([[ 0.2504, -0.2141, -0.1906, -0.0511, -0.0194,  0.1235,  0.0415,  0.1609,
           0.0399,  0.0081],
         [ 0.1363, -0.2767, -0.0794,  0.0048,  0.1427,  0.1200, -0.0196,  0.0930,
          -0.1435,  0.0266]], grad_fn=<AddmmBackward>))

`nn.Sequential` 定义了一种特殊的 `Module`

## 自定义块

In [15]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

## 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [19]:
net = MLP()
net(X)
# net(X) 实际上调用了 net.__call__(), 而 __call()__ 调用了 forward

tensor([[-0.3570, -0.0296, -0.1181,  0.1323, -0.0276,  0.1250,  0.1325, -0.0912,
          0.0574, -0.0617],
        [-0.2723, -0.0919, -0.0583,  0.1108, -0.0394, -0.0113,  0.1171, -0.1127,
          0.0608, -0.1126]], grad_fn=<AddmmBackward>)

## 顺序块

In [22]:
# 实现和 nn.Sequential 一样的功能
class MySequential(nn.Module):
    def __init__(self, *args): # *args 是收集参数，相当于把若干个参数打包成一个来传入
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0372,  0.1155,  0.3315, -0.1325,  0.1238, -0.0546,  0.0012, -0.1080,
         -0.0551,  0.1643],
        [ 0.1194,  0.1517,  0.3125, -0.1038,  0.2493,  0.0157,  0.0747,  0.0066,
         -0.1377, -0.0362]], grad_fn=<AddmmBackward>)

## 在正向传播函数中执行代码

In [26]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X): # 自定义前向传播函数
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.3095, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [27]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1092, grad_fn=<SumBackward0>)

---

## 参数管理
我们首先关注具有单隐藏层的多层感知机

In [29]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0568],
        [0.0988]], grad_fn=<AddmmBackward>)

**参数访问**

In [31]:
print(net[2].state_dict()) 
# net[0] -> nn.Linear(4, 8) ;  net[1]- > nn.ReLU()  ; net[2] -> nn.Linear(8, 1)

OrderedDict([('weight', tensor([[-0.2235, -0.0193,  0.2738,  0.1471,  0.0151, -0.3304, -0.2461,  0.2094]])), ('bias', tensor([0.2087]))])


**目标参数**

In [32]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2087], requires_grad=True)
tensor([0.2087])


In [33]:
net[2].weight.grad == None

True

**一次性访问所有参数**

In [34]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [35]:
net.state_dict()['2.bias'].data

tensor([0.2087])

## 从嵌套块收集参数

In [38]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
        
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2511],
        [0.2511]], grad_fn=<AddmmBackward>)

我们已经设计了 `rgnet` 这个网络，让我们看看他是如何组织的

In [39]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 内置初始化 
刚才我们在讲如何访问权重，偏移；接下来看看如何初始化这些参数

In [41]:
def init_normal(m): # m 代表一个 module
    if type(m) == nn.Linear: # 如果这个 module 是一个线性层（全连接层）
        nn.init.normal_(m.weight, mean=0, std=0.01) # 如果是一个全连接层，就做一个均值为 0，方差为 0.01 的初始化
        nn.init.zeros_(m.bias) # 对偏差赋 0 

net.apply(init_normal) # apply 方法相当于对 net 中的 Layer 进行遍历 init_normal 方法（会递归调用，直到所有层都初始化）
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0143,  0.0015, -0.0125, -0.0057]), tensor(0.))

In [42]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) # 将 weight 初始化为 一个常量为 1 的值
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法（可以对不同的层 apply 不同的事情）

In [46]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) # 对 weight 做 xvaier 的初始化

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 对 weight 初始化为 一个常量为 42 的值

# net 上面已经定义过，有三层，Linear(4,8)  ReLU()  Linear(8,1)
net[0].apply(xavier) # 对第一层用 xavier 这个初始化
net[2].apply(init_42) # 对第三层用 inir_42 这个初始化
print(net[0].weight.data[0])
print(net[2].weight.data)
print(net)

tensor([-0.3851,  0.2236,  0.4877,  0.4356])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


自定义初始化

In [48]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  7.4602,  8.6357, -8.8766],
        [-7.9709, -0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward>)

In [49]:
# 其实可以直接暴力对数值替换也可以
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  8.4602,  9.6357, -7.8766])

参数绑定

In [50]:
# 在一些层之间 share 一些 parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


---

## 自定义层
构造一个没有任何参数的自定义层

In [53]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到构建更复杂的模型中

In [55]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-2.7940e-09, grad_fn=<MeanBackward0>)

带参数的图层

In [57]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-1.7850,  0.7576, -1.1949],
        [ 0.0307,  2.3951, -0.1846],
        [-0.5598,  0.3427,  0.1086],
        [-1.2969,  0.0355, -0.4974],
        [ 0.2240,  0.8727, -1.5616]], requires_grad=True)

使用自定义层直接执行正向传播函数

In [58]:
dense(torch.rand(2, 5))

tensor([[1.0641, 2.9986, 0.0000],
        [0.0574, 2.9521, 0.0000]])

使用自定义层构建模型

In [59]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[8.4780],
        [6.7824]])

---

## 读写文件
加载和保存张量

In [60]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后把它们读回内存

In [62]:
y  = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取从字符串映射到张量的字典

In [63]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [67]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
Y

tensor([[-0.4619,  0.6240,  0.0030,  0.0533,  0.0621,  0.2536,  0.2018,  0.5331,
         -0.1941,  0.1314],
        [-0.0534,  0.5983, -0.0504,  0.0876, -0.4686, -0.2234, -0.2682,  0.2525,
          0.0601,  0.2416]], grad_fn=<AddmmBackward>)

将模型的参数存储为一个叫做“mlp.params”的文件

In [68]:
torch.save(net.state_dict(), 'mlp.params')

实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数

In [69]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [71]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])